# showus

In [ ]:
#default_exp showus

In [ ]:
! pip install /kaggle/input/nlp-packages/datasets/datasets/fsspec-2021.4.0-py3-none-any.whl
! pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
! pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
! pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
! pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

Processing /kaggle/input/nlp-packages/datasets/datasets/fsspec-2021.4.0-py3-none-any.whl
  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.8.7
    Uninstalling fsspec-0.8.7:
      Successfully uninstalled fsspec-0.8.7
Looking in links: file:///kaggle/input/coleridge-packages/packages/datasets
Processing /kaggle/input/coleridge-packages/packages/datasets/datasets-1.5.0-py3-none-any.whl
Processing /kaggle/input/coleridge-packages/packages/datasets/tqdm-4.49.0-py2.py3-none-any.whl
Processing /kaggle/input/coleridge-packages/packages/datasets/xxhash-2.0.0-cp37-cp37m-manylinux2010_x86_64.whl
Processing /kaggle/input/coleridge-packages/packages/datasets/huggingface_hub-0.0.7-py3-none-any.whl
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0
Processing /kaggle/input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
Processing /kaggle/input/coleridge-packages/tokenizers-0.10.

In [ ]:
#export
import os, shutil
from tqdm import tqdm
from pathlib import Path
import itertools
from functools import partial
import re
import json
import random
import numpy as np
import pandas as pd
import torch
import transformers, seqeval
from transformers import AutoTokenizer, DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset, ClassLabel, load_metric

# Utilities

In [ ]:
#export
Path.ls = lambda pth: list(pth.iterdir())

# Data I/O

In [ ]:
#export
def load_train_meta(pth, group_id=True):
    df = pd.read_csv(pth)
    if group_id:
        df = df.groupby('Id').agg({'pub_title': 'first', 'dataset_title': '|'.join, 
                                   'dataset_label': '|'.join, 'cleaned_label': '|'.join}).reset_index()
    return df

In [ ]:
pth = Path('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
df = load_train_meta(pth, group_id=True)
df_nogroup = load_train_meta(pth, group_id=False)
print(len(df), len(df_nogroup))
dup_ids = df_nogroup[df_nogroup.Id.duplicated()].Id.unique()
print(df[df.Id.isin(dup_ids)].dataset_label.values[-10:])

14316 19661
['Baltimore Longitudinal Study of Aging (BLSA)|Baltimore Longitudinal Study of Aging'
 'Beginning Postsecondary Students Longitudinal Study|Education Longitudinal Study|Beginning Postsecondary Students'
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 'Baltimore Longitudinal Study of Aging (BLSA)|Baltimore Longitudinal Study of Aging'
 "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
 'Beginning Postsecondary Student|Beginning Postsecondary Students']


In [ ]:
#export
def load_papers(dir_json, paper_ids):
    '''
    Load papers into a dictionary.
    
    `papers`: 
        {''}
    '''
    
    papers = {}
    for paper_id in paper_ids:
        with open(f'{dir_json}/{paper_id}.json', 'r') as f:
            paper = json.load(f)
            papers[paper_id] = paper
    return papers

In [ ]:
df = load_train_meta('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv', group_id=True).iloc[-10:]
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/train/', df.Id)
print(type(papers))
print(
    papers[ np.random.choice(df.Id.values) ][0]
)

<class 'dict'>
{'section_title': 'Abstract', 'text': 'Mathematics, as no other school subject, evokes conflicting emotions and contradictory attitudes -from "the gate to a career" and "the queen of science" to the widespread acceptance of mathematical ignorance in society. The process of studying mathematics requires systematic work and patience, as mathematical knowledge has a cumulative nature. In the case of mathematics education, some students abandon mathematics at quite early levels of education and begin to consider themselves "humanists", which results in serious consequences for future educational and career choices. In this paper, I propose a description of the process of escaping from mathematics in the context of students\' perceptions of this subject, using the results of two studies -one qualitative and the other quantitative.'}


In [ ]:
# df.dataset_label.iloc[3]

# paper = papers[df.Id.iloc[4]]
# ['\n'.join([section['section_title'], section['text']]).split() for section in paper if section['text']][2]

In [ ]:
#export
def load_sample_text(jpth):
    sections = json.loads(jpth.read_text())
    text = '\n'.join(section['text'] for section in sections)
    return text

In [ ]:
jpths_trn = Path('/kaggle/input/coleridgeinitiative-show-us-the-data/train/').ls()
print(load_sample_text(jpths_trn[0])[:1_000])

The International Standard Classification of Education, known by its acronym ISCED, was developed by the United Nations Educational, Scientific, and Cultural Organization during the late 1960s and 1970s. ISCED was implemented in 1976 and is the recognized international standard for reporting and interpreting education program data. Creating a U.S. crosswalk to this system has been a goal of the National Center for Education Statistics and the Office of Research since the late 197,,s, when the National Institute of Education (the predecessor agency to the Office of Educational Research and Improvement) began exploring the idea. The design and implementation of a workable crosswalk, however, awaited the advent of changes to the Classification of Instructional Programs (CIP) system. The 1990 revision of the CIP system laid the foundation for a workable international crosswalk. Adoption of the National Education Goals set global consciousness and international educational comparisons firml

# Data processing

In [ ]:
#export
def clean_training_text(txt, lower=False, total_clean=False):
    """
    similar to the default clean_text function but without lowercasing.
    """
    txt = str(txt).lower() if lower else str(txt)
    txt = re.sub('[^A-Za-z0-9]+', ' ', txt).strip()
    if total_clean:
        txt = re.sub(' +', ' ', txt)
    return txt

In [ ]:
print(clean_training_text('@kaggle This competition awards $90,000!!!!.'))
print(clean_training_text('HoPKLd + 7 ! 11,002', total_clean=True, lower=True))

kaggle This competition awards 90 000
hopkld 7 11 002


In [ ]:
#export
def shorten_sentences(sentences, max_length=64, overlap=20):
    '''
    Args:
        sentences (list): List of sentences.
        max_length (int): Maximum number of words allowed for each sentence.
        overlap (int): If a sentence exceeds `max_length`, we split it to multiple sentences with 
            this amount of overlapping.
    '''
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > max_length:
            for p in range(0, len(words), max_length - overlap):
                short_sentences.append(' '.join(words[p:p+max_length]))
        else:
            short_sentences.append(sentence)
    return short_sentences

In [ ]:
jpths_trn = Path('/kaggle/input/coleridgeinitiative-show-us-the-data/train/').ls()
sentences = load_sample_text(jpths_trn[0]).split('.')[:2]
short_sentences = shorten_sentences(sentences, max_length=10, overlap=2)
print('Before:', sentences)
print()
print('After:', short_sentences)

Before: ['The International Standard Classification of Education, known by its acronym ISCED, was developed by the United Nations Educational, Scientific, and Cultural Organization during the late 1960s and 1970s', ' ISCED was implemented in 1976 and is the recognized international standard for reporting and interpreting education program data']

After: ['The International Standard Classification of Education, known by its acronym', 'its acronym ISCED, was developed by the United Nations Educational,', 'Nations Educational, Scientific, and Cultural Organization during the late 1960s', 'late 1960s and 1970s', 'ISCED was implemented in 1976 and is the recognized international', 'recognized international standard for reporting and interpreting education program data', 'program data']


In [ ]:
#export
def find_sublist(big_list, small_list):
    all_positions = []
    for i in range(len(big_list) - len(small_list) + 1):
        if small_list == big_list[i:i+len(small_list)]:
            all_positions.append(i)
    
    return all_positions

In [ ]:
big_list = ['If', 'the', 'thing', 'above', 'is', 'below', 'that', 'thing', 'which', 'is',
            'not', 'as', 'high', 'up', 'on', 'the', 'thing', 'above', 'when', 'it', 'is', 
            'underneath', 'them.']
small_list = ['the', 'thing', 'above']

find_sublist(big_list, small_list)

[1, 15]

# Named Entity Recognition

In [ ]:
#export
def get_ner_classlabel():
    '''
    Labels for named entity recognition.
        'O': Token not part of a phrase that mentions a dataset.
        'I': Intermediate token of a phrase mentioning a dataset.
        'B': First token of a phrase mentioning a dataset.
    '''
    return ClassLabel(names=['O', 'I', 'B'])

In [ ]:
classlabel = get_ner_classlabel()
print(classlabel)
print(classlabel.str2int(['I', 'O', 'B']), classlabel.str2int('I'))
print(classlabel.int2str(2), classlabel.int2str([2, 1, 0]))

ClassLabel(num_classes=3, names=['O', 'I', 'B'], names_file=None, id=None)
[1, 0, 2] 1
B ['B', 'I', 'O']


In [ ]:
#export
def tag_sentence(sentence, labels, classlabel=None): 
    '''
    requirement: both sentence and labels are already cleaned
    '''
    sentence_words = sentence.split()
    
    if labels is not None and any(re.findall(f'\\b{label}\\b', sentence)
                                  for label in labels): # positive sample
        nes = [classlabel.str2int('O')] * len(sentence_words)
        for label in labels:
            label_words = label.split()

            all_pos = find_sublist(sentence_words, label_words)
            for pos in all_pos:
                nes[pos] = classlabel.str2int('B')
                for i in range(pos+1, pos+len(label_words)):
                    nes[i] = classlabel.str2int('I')

        return True, list(zip(sentence_words, nes))
        
    else: # negative sample
        nes = [classlabel.str2int('O')] * len(sentence_words)
        return False, list(zip(sentence_words, nes))

In [ ]:
sentence = ("The International Standard Classification of Education, known by its acronym ISCED, "
            "was developed by the United Nations Educational, "
            "Scientific, and Cultural Organization during the late 1960s and 1970s")
labels = ['The International', 'Cultural Organization', 'United Nations Educational']

sentence = clean_training_text(sentence)
labels = [clean_training_text(label) for label in labels]
classlabel = get_ner_classlabel()
found_any, token_tags = tag_sentence(sentence, labels, classlabel=classlabel)

print('A label is found in the sentence:', found_any)
print('(token, tag) pairs:')
print(token_tags)

A label is found in the sentence: True
(token, tag) pairs:
[('The', 2), ('International', 1), ('Standard', 0), ('Classification', 0), ('of', 0), ('Education', 0), ('known', 0), ('by', 0), ('its', 0), ('acronym', 0), ('ISCED', 0), ('was', 0), ('developed', 0), ('by', 0), ('the', 0), ('United', 2), ('Nations', 1), ('Educational', 1), ('Scientific', 0), ('and', 0), ('Cultural', 2), ('Organization', 1), ('during', 0), ('the', 0), ('late', 0), ('1960s', 0), ('and', 0), ('1970s', 0)]


In [ ]:
#export
def extract_sentences(paper, sentence_definition='sentence'):
    if sentence_definition == 'sentence':
        sentences = set(clean_training_text(sentence) 
                        for sec in paper for sentence in sec['text'].split('.') if sec['text'])
    elif sentence_definition == 'section':
        sentences = set(clean_training_text(sec['section_title'] + '\n' + sec['text']) 
                        for sec in paper if sec['text'])
    return sentences

In [ ]:
df = load_train_meta('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv', group_id=True).iloc[100:110]
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/train', df.Id)
paper = papers[df.Id.iloc[3]]
print('Sentence definition = normal sentence')
sentences = extract_sentences(paper, sentence_definition='sentence')
print(len(sentences), list(sentences)[:10], end='\n\n')
print('Sentence definition = paper section')
sentences = extract_sentences(paper, sentence_definition='section')
print(len(sentences), list(sentences)[:2], end='\n\n')

Sentence definition = normal sentence
302 ['', 'Charter schools are public schools that are founded by community leaders and parents', 'Section 6 summarizes the main findings addresses limitations and offers suggestions for future research', 'g', 'Card and Krueger 1992 and further educational attainment e', 'One additional factor that might affect test performance is the degree of urbanization', 'We explore this question most appropriately addressed from a longitudinal perspective in the next two subsections', 'First we ignore all non charter school intra system choices', 'The term traditional is not meant to suggest that charter schools necessarily adopt an atypical curriculum', 'Finally Greene and Forster 2002 report gains to competition as measured by an index of the distance between traditional schools and charters']

Sentence definition = paper section
12 ['Cross section regressions We begin by exploring cross section regression models Beyond the potential inconsistency from the u

In [ ]:
#export
def get_paper_ner_data(paper, labels, classlabel=None,
                       sentence_definition='sentence', max_length=64, overlap=20):
    '''
    Get NER data for a paper.
    '''
    labels = [clean_training_text(label) for label in labels]
    sentences = extract_sentences(paper, sentence_definition=sentence_definition)
    sentences = shorten_sentences(sentences, max_length=max_length, overlap=overlap) 
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars

    cnt_pos, cnt_neg, ner_data = 0, 0, []
    for sentence in sentences:
        is_positive, tags = tag_sentence(sentence, labels, classlabel=classlabel)
        if is_positive:
            cnt_pos += 1
            ner_data.append(tags)
        elif any(word in sentence.lower() for word in ['data', 'study']): 
            ner_data.append(tags)
            cnt_neg += 1    
    return cnt_pos, cnt_neg, ner_data

In [ ]:
df = load_train_meta('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv', group_id=True).iloc[230:240]
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/train/', df.Id)
classlabel = get_ner_classlabel()

idx = 8
paper = papers[df.Id.iloc[idx]]
labels = df.dataset_label.iloc[idx].split('|')
cnt_pos, cnt_neg, ner_data = get_paper_ner_data(paper, labels, classlabel=classlabel,
                                                sentence_definition='section', max_length=512, overlap=20)
print(cnt_pos, cnt_neg)
print([len(sec) for sec in ner_data])
print(ner_data[-1])

9 1
[101, 512, 512, 49, 303, 512, 512, 512, 512, 259]
[('as', 0), ('potential', 0), ('benefits', 0), ('implement', 0), ('lifestyle', 0), ('changes', 0), ('planning', 0), ('for', 0), ('the', 0), ('future', 0), ('eligibility', 0), ('for', 0), ('intervention', 0), ('trials', 0), ('Finally', 0), ('respondents', 0), ('questioned', 0), ('the', 0), ('validity', 0), ('of', 0), ('amyloid', 0), ('imaging', 0), ('results', 0), ('and', 0), ('called', 0), ('for', 0), ('more', 0), ('longitudinal', 0), ('data', 0), ('especially', 0), ('in', 0), ('cognitively', 0), ('normal', 0), ('individuals', 0), ('regarding', 0), ('risk', 0), ('of', 0), ('development', 0), ('of', 0), ('AD', 0), ('dementia', 0), ('Other', 0), ('topics', 0), ('respondents', 0), ('cited', 0), ('in', 0), ('need', 0), ('of', 0), ('research', 0), ('were', 0), ('solving', 0), ('ongoing', 0), ('technical', 0), ('issues', 0), ('related', 0), ('to', 0), ('the', 0), ('validity', 0), ('of', 0), ('amyloid', 0), ('imaging', 0), ('data', 0), ('d

In [ ]:
#export
def get_ner_data(papers, df=None, classlabel=None, shuffle=True, 
                 sentence_definition='sentence', max_length=64, overlap=20):
    '''
    Args:
        papers (dict): Like that returned by `load_papers`.
        df (pd.DataFrame): Competition's train.csv or a subset of it.
    '''
    cnt_pos, cnt_neg = 0, 0 
    ner_data = []

    tqdm._instances.clear()
    pbar = tqdm(total=len(df))
    for i, id, dataset_label in df[['Id', 'dataset_label']].itertuples():
        paper = papers[id]
        labels = dataset_label.split('|')
                
        cnt_pos_, cnt_neg_, ner_data_ = get_paper_ner_data(
            paper, labels, classlabel=classlabel, 
            sentence_definition=sentence_definition, max_length=max_length, overlap=overlap)
        cnt_pos += cnt_pos_
        cnt_neg += cnt_neg_
        ner_data.extend(ner_data_)

        pbar.update(1)
        pbar.set_description(f"Training data size: {cnt_pos} positives + {cnt_neg} negatives")

    if shuffle:
        random.shuffle(ner_data)
    return cnt_pos, cnt_neg, ner_data

In [ ]:
%%time
df = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv').iloc[:100]
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/train/', df.Id)
classlabel = get_ner_classlabel()
cnt_pos, cnt_neg, ner_data = get_ner_data(papers, df, classlabel=classlabel, shuffle=False,
                                          sentence_definition='sentence', max_length=64, overlap=20)
print(f'Postive count: {cnt_pos}.   Negative count: {cnt_neg}')
print(ner_data[250])

Training data size: 168 positives + 2574 negatives: 100%|██████████| 100/100 [00:00<00:00, 120.39it/s]

Postive count: 168.   Negative count: 2574
[('Likewise', 0), ('there', 0), ('is', 0), ('a', 0), ('familiarity', 0), ('with', 0), ('a', 0), ('variety', 0), ('of', 0), ('studies', 0), ('that', 0), ('provide', 0), ('specific', 0), ('data', 0), ('on', 0), ('the', 0), ('academic', 0), ('achievement', 0), ('of', 0), ('Catholic', 0), ('school', 0), ('students', 0)]
CPU times: user 893 ms, sys: 47.9 ms, total: 941 ms
Wall time: 1.31 s


In [ ]:
%%time
df = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv').iloc[:100]
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/train/', df.Id)
classlabel = get_ner_classlabel()
cnt_pos, cnt_neg, ner_data = get_ner_data(papers, df, classlabel=classlabel, shuffle=False, 
                                          sentence_definition='section', max_length=512, overlap=20)
print(f'Postive count: {cnt_pos}.   Negative count: {cnt_neg}')
print(ner_data[3])

Training data size: 171 positives + 910 negatives: 100%|██████████| 100/100 [00:00<00:00, 136.79it/s]

Postive count: 171.   Negative count: 910
[('WWC', 0), ('Single', 0), ('Study', 0), ('Review', 0), ('What', 0), ('did', 0), ('the', 0), ('study', 0), ('find', 0), ('The', 0), ('study', 0), ('reported', 0), ('and', 0), ('the', 0), ('WWC', 0), ('confirmed', 0), ('that', 0), ('dual', 0), ('enrollment', 0), ('programs', 0), ('significantly', 0), ('increased', 0), ('the', 0), ('likelihood', 0), ('of', 0), ('attaining', 0), ('a', 0), ('any', 0), ('college', 0), ('degree', 0), ('and', 0), ('b', 0), ('a', 0), ('bachelor', 0), ('s', 0), ('degree', 0), ('The', 0), ('study', 0), ('reported', 0), ('on', 0), ('the', 0), ('impact', 0), ('of', 0), ('dual', 0), ('enrollment', 0), ('programs', 0), ('for', 0), ('first', 0), ('generation', 0), ('college', 0), ('students', 0), ('students', 0), ('whose', 0), ('parents', 0), ('had', 0), ('some', 0), ('college', 0), ('students', 0), ('whose', 0), ('parents', 0), ('had', 0), ('a', 0), ('Bachelor', 0), ('s', 0), ('degree', 0), ('and', 0), ('students', 0), ('wi

In [ ]:
#export
def write_ner_json(ner_data, pth=Path('train_ner.json')):
    with open(pth, 'w') as f:
        for row in ner_data:
            words, nes = list(zip(*row))
            row_json = {'tokens' : words, 'ner_tags' : nes}
            json.dump(row_json, f)
            f.write('\n')    

In [ ]:
ner_data = [
    [('There', 0), ('is', 0), ('no', 0), ('dataset', 0), ('here', 0)], 
    [('Load', 0), ('the', 0), ('UN', 2), ('Trade', 1), ('Development', 1), ('into', 0), ('view', 0)]
]
write_ner_json(ner_data, pth=Path('/kaggle/tmp_ner.json'))
! cat /kaggle/tmp_ner.json

{"tokens": ["There", "is", "no", "dataset", "here"], "ner_tags": [0, 0, 0, 0, 0]}
{"tokens": ["Load", "the", "UN", "Trade", "Development", "into", "view"], "ner_tags": [0, 0, 2, 1, 1, 0, 0]}


In [ ]:
#export
def load_ner_datasets(data_files=None):
    datasets = load_dataset('json', data_files=data_files)
    classlabel = get_ner_classlabel()
    for split, dataset in datasets.items():
        dataset.features['ner_tags'].feature = classlabel
    return datasets

In [ ]:
datasets = load_ner_datasets(data_files={'train':'/kaggle/tmp_ner.json', 'valid':'/kaggle/tmp_ner.json'})
print(datasets['valid'].features)
print()
print(datasets['train'][1])

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-5c75d4e550e0f5d5/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=3, names=['O', 'I', 'B'], names_file=None, id=None), length=-1, id=None)}

{'tokens': ['Load', 'the', 'UN', 'Trade', 'Development', 'into', 'view'], 'ner_tags': [0, 0, 2, 1, 1, 0, 0]}


In [ ]:
#export
def create_tokenizer(model_checkpoint='bert-base-cased'):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
    return tokenizer

In [ ]:
tokenizer = create_tokenizer(model_checkpoint='bert-base-cased')
print(
    tokenizer("A smattering of people descended from the midday boat on Monday."))
print()
print(
    tokenizer("Giglio boasts several pristine bays with crystal clear water".split(), is_split_into_words=True)
)


{'input_ids': [101, 138, 188, 21943, 9930, 1104, 1234, 9026, 1121, 1103, 2286, 6194, 3499, 1113, 6356, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

{'input_ids': [101, 144, 6512, 9436, 24372, 1317, 185, 12937, 2042, 15520, 1114, 8626, 2330, 1447, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
#export
def tokenize_and_align_labels(examples, tokenizer=None, label_all_tokens=True):
    '''
    Adds a new field called 'labels' that are the NER tags to the tokenized input.
    
    Args:
        tokenizer (transformers.AutoTokenizer): Tokenizer.
        examples (datasets.arrow_dataset.Dataset): Dataset.
        label_all_tokens (bool): If True, all sub-tokens are given the same tag as the 
            first sub-token, otherwise all but the first sub-token are given the tag
            -100.
    '''
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
datasets = load_ner_datasets(data_files={'train':'/kaggle/tmp_ner.json', 'valid':'/kaggle/tmp_ner.json'})
tokenizer = create_tokenizer(model_checkpoint='bert-base-cased')

print(tokenize_and_align_labels(datasets['train'][:], tokenizer, label_all_tokens=True), end='\n\n')

tokenized_datasets = datasets.map(
    partial(tokenize_and_align_labels, tokenizer=tokenizer, label_all_tokens=True), batched=True)

print(tokenized_datasets['train'][:], end='\n\n')
print(tokenized_datasets['valid'][:])
print(tokenized_datasets['train'].features)

{'input_ids': [[101, 1247, 1110, 1185, 2233, 9388, 1303, 102], [101, 10605, 3556, 1103, 7414, 5820, 3273, 1154, 2458, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 2, 1, 1, 0, 0, -100]]}




{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'input_ids': [[101, 1247, 1110, 1185, 2233, 9388, 1303, 102], [101, 10605, 3556, 1103, 7414, 5820, 3273, 1154, 2458, 102]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 2, 1, 1, 0, 0, -100]], 'ner_tags': [[0, 0, 0, 0, 0], [0, 0, 2, 1, 1, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'tokens': [['There', 'is', 'no', 'dataset', 'here'], ['Load', 'the', 'UN', 'Trade', 'Development', 'into', 'view']]}

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'input_ids': [[101, 1247, 1110, 1185, 2233, 9388, 1303, 102], [101, 10605, 3556, 1103, 7414, 5820, 3273, 1154, 2458, 102]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, 2, 1, 1, 0, 0, -100]], 'ner_tags': [[0, 0, 0, 0, 0], [0, 0, 2, 1, 1, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'tokens': [['There', 'is', 'no', 'dataset', 'he

In [ ]:
metric = load_metric('seqeval')

predictions = [['O', 'O', 'B', 'I', 'I', 'O']]
references = [['B', 'I', 'B', 'O', 'O', 'O']]
metric.compute(predictions=predictions, references=references)

{'_': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2},
 'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.3333333333333333}

In [ ]:
#export
def jaccard_similarity(s1, s2):
    l1 = set(s1.split(" "))
    l2 = set(s2.split(" "))
    intersection = len(list(l1.intersection(l2)))
    union = (len(l1) + len(l2)) - intersection
    return float(intersection) / union

In [ ]:
jaccard_similarity('USGS Frog Counts Data', 'USGA Croc Counts Data') == 1 / 3

True

In [ ]:
#export
def compute_metrics(p, metric=None, label_list=None):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
classlabel = get_ner_classlabel()

batch_size = 8
max_sentence_length = 30

predictions = np.random.randn(batch_size, max_sentence_length, classlabel.num_classes)
label_ids = np.random.randint(low=0, high=classlabel.num_classes, 
                              size=(batch_size, max_sentence_length), dtype=np.int16)

p = (predictions, label_ids)
metric = load_metric('seqeval')
compute_metrics(p, metric=metric, label_list=classlabel.names)

{'precision': 0.24786324786324787,
 'recall': 0.26605504587155965,
 'f1': 0.25663716814159293,
 'accuracy': 0.38333333333333336}

## NER training

In [ ]:
classlabel = get_ner_classlabel()
train_meta = load_train_meta('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv').iloc[:4]
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/train', train_meta.Id)

valid_cutoff = int(.50 * len(train_meta))
valid_meta = train_meta.iloc[:valid_cutoff].reset_index(drop=True)
train_meta = train_meta.iloc[valid_cutoff:].reset_index(drop=True)

train_cnt_pos, train_cnt_neg, train_ner_data = get_ner_data(
    papers, df=train_meta, classlabel=classlabel, sentence_definition='section', max_length=512, overlap=20)
valid_cnt_pos, valid_cnt_neg, valid_ner_data = get_ner_data(
    papers, df=valid_meta, classlabel=classlabel, sentence_definition='section', max_length=512, overlap=20)
print(f'Train.  Positive count: {train_cnt_pos}.  Negative count: {train_cnt_neg}.')
print(f'Valid.  Positive count: {valid_cnt_pos}.  Negative count: {valid_cnt_neg}.')

write_ner_json(train_ner_data, pth='train_ner.json')
write_ner_json(valid_ner_data, pth='valid_ner.json')

Training data size: 2 positives + 23 negatives: 100%|██████████| 2/2 [00:00<00:00, 138.17it/s]

Train.  Positive count: 8.  Negative count: 24.
Valid.  Positive count: 2.  Negative count: 23.


In [ ]:
datasets = load_ner_datasets(data_files={'train':'train_ner.json', 'valid':'valid_ner.json'})

model_checkpoint = 'distilbert-base-cased'
tokenizer = create_tokenizer(model_checkpoint)
tokenized_datasets = datasets.map(
    partial(tokenize_and_align_labels, tokenizer=tokenizer, label_all_tokens=True), batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=classlabel.num_classes)

metric = load_metric('seqeval')

args = TrainingArguments(output_dir='test_ner', num_train_epochs=2, 
                         learning_rate=2e-5, weight_decay=0.01,
                         per_device_train_batch_size=16, per_device_eval_batch_size=16,
                         evaluation_strategy='epoch', logging_steps=4, report_to='none', 
                         save_strategy='epoch', save_total_limit=6)

trainer = Trainer(model=model, args=args, 
                  train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['valid'], 
                  data_collator=data_collator, tokenizer=tokenizer, 
                  compute_metrics=partial(compute_metrics, metric=metric, label_list=classlabel.names))

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d57444ea6d3c2668/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
1,No log,0.658076,0.000000,0.000000,0.000000,0.989687,19.763200,1.265000
2,0.763700,0.528109,0.000000,0.000000,0.000000,0.996298,19.617000,1.274000


TrainOutput(global_step=4, training_loss=0.7636871337890625, metrics={'train_runtime': 221.5724, 'train_samples_per_second': 0.018, 'total_flos': 12817508401152.0, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 3299872768, 'train_mem_cpu_peaked_delta': 6719229952})

In [ ]:
! ls -lrt test_ner

total 8
drwxr-xr-x 2 root root 4096 May 29 09:55 checkpoint-2
drwxr-xr-x 2 root root 4096 May 29 09:57 checkpoint-4


In [ ]:
args = TrainingArguments(output_dir='test_ner', num_train_epochs=4, 
                         learning_rate=2e-5, weight_decay=0.01,
                         per_device_train_batch_size=16, per_device_eval_batch_size=16,
                         evaluation_strategy='epoch', logging_steps=4, report_to='none', 
                         save_strategy='epoch', save_total_limit=6)

trainer = Trainer(model=model, args=args, 
                  train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['valid'], 
                  data_collator=data_collator, tokenizer=tokenizer, 
                  compute_metrics=partial(compute_metrics, metric=metric, label_list=classlabel.names))
trainer.train(resume_from_checkpoint='/kaggle/working/test_ner/checkpoint-4/')

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
3,0.763700,0.464731,0.000000,0.000000,0.000000,0.996959,19.545600,1.279000
4,0.452500,0.403027,0.000000,0.000000,0.000000,0.997223,19.790900,1.263000


TrainOutput(global_step=8, training_loss=0.22624197602272034, metrics={'train_runtime': 224.2963, 'train_samples_per_second': 0.036, 'total_flos': 25635016802304.0, 'epoch': 4.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 3775193088, 'train_mem_cpu_peaked_delta': 4076724224})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.40302741527557373,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_accuracy': 0.9972233240777469,
 'eval_runtime': 19.9006,
 'eval_samples_per_second': 1.256,
 'epoch': 4.0,
 'eval_mem_cpu_alloc_delta': 69632,
 'eval_mem_cpu_peaked_delta': 0}

In [ ]:
predictions, label_ids, _ = trainer.predict(tokenized_datasets['valid'])
predictions = np.argmax(predictions, axis=2)

true_predictions = [[classlabel.names[p] for p, i in zip(prediction, label_id) if i != -100]
                    for prediction, label_id in zip(predictions, label_ids)]

true_labels = [[classlabel.names[i] for i in label_id if i != -100] for label_id in label_ids]

print(true_predictions[0])
print(true_labels[0])

results = metric.compute(predictions=true_predictions, references=true_labels)
print(results)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

## NER inference

**Turn off the Internet here**

In [ ]:
#export
def get_ner_inference_data(papers, sample_submission, classlabel=None, 
                           sentence_definition='sentence', max_length=64, overlap=20):
    '''
    Args:
        papers (dict): Each list in this dictionary consists of the section of a paper.
        sample_submission (pd.DataFrame): Competition 'sample_submission.csv'.
    Returns:
        test_rows (list): Each list in this list is of the form: 
             [('goat', 0), ('win', 0), ...] and represents a sentence.  
        paper_length (list): Number of sentences in each paper.
    '''
    test_rows = [] 
    paper_length = [] 

    for paper_id in sample_submission['Id']:
        paper = papers[paper_id]

        sentences = extract_sentences(paper, sentence_definition=sentence_definition)
        sentences = shorten_sentences(sentences, max_length=max_length, overlap=overlap)
        sentences = [sentence for sentence in sentences if len(sentence) > 10] 
        sentences = [sentence for sentence in sentences if any(word in sentence.lower() for word in ['data', 'study'])]

        for sentence in sentences:
            sentence_words = sentence.split()
            dummy_tags = [classlabel.str2int('O')]*len(sentence_words)
            test_rows.append(list(zip(sentence_words, dummy_tags)))

        paper_length.append(len(sentences))

    print(f'total number of sentences: {len(test_rows)}')
    return test_rows, paper_length

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/test', sample_submission.Id)
classlabel = get_ner_classlabel()
test_rows, paper_length = get_ner_inference_data(papers, sample_submission, classlabel=classlabel, 
                                                 sentence_definition='section', max_length=512, overlap=20)
print(test_rows[1])
print(paper_length)

total number of sentences: 105
[('Methods', 0), ('and', 0), ('Materials', 0), ('COGENT', 0), ('is', 0), ('an', 0), ('international', 0), ('GWAS', 0), ('collaboration', 0), ('formed', 0), ('to', 0), ('conduct', 0), ('genetic', 0), ('analyses', 0), ('of', 0), ('g', 0), ('and', 0), ('related', 0), ('neurocognitive', 0), ('processes', 0), ('in', 0), ('healthy', 0), ('individuals', 0), ('Donohoe', 0), ('et', 0), ('al', 0), ('2012', 0), ('Though', 0), ('common', 0), ('GWAS', 0), ('markers', 0), ('have', 0), ('been', 0), ('proposed', 0), ('to', 0), ('account', 0), ('for', 0), ('30', 0), ('or', 0), ('more', 0), ('of', 0), ('the', 0), ('variance', 0), ('in', 0), ('general', 0), ('intelligence', 0), ('in', 0), ('adults', 0), ('individual', 0), ('SNPs', 0), ('only', 0), ('contribute', 0), ('a', 0), ('small', 0), ('fraction', 0), ('of', 0), ('the', 0), ('variance', 0), ('to', 0), ('the', 0), ('heritability', 0), ('of', 0), ('g', 0), ('due', 0), ('to', 0), ('extreme', 0), ('polygenicity', 0), ('Mar

In [ ]:
#export
def ner_predict(test_rows, tokenizer=None, model=None, metric=None):
    classlabel = get_ner_classlabel()
    
    write_ner_json(test_rows, pth='test_ner.json')
    datasets = load_ner_datasets(data_files={'test':'test_ner.json'})
    print('Tokenizing testset...')
    tokenized_datasets = datasets.map(
        partial(tokenize_and_align_labels,tokenizer=tokenizer, label_all_tokens=True), 
        batched=True) 

    print('Creating data collator...')
    data_collator = DataCollatorForTokenClassification(tokenizer)
    print('Creating (dummy) training arguments...')
    args = TrainingArguments(output_dir='test_ner', num_train_epochs=3, 
                             learning_rate=2e-5, weight_decay=0.01,
                             per_device_train_batch_size=16, per_device_eval_batch_size=16,
                             evaluation_strategy='epoch', logging_steps=4, report_to='none', 
                             save_strategy='epoch', save_total_limit=6)

    print('Creating trainer...')
    trainer = Trainer(model=model, args=args, 
                      train_dataset=tokenized_datasets['test'], eval_dataset=tokenized_datasets['test'], 
                      data_collator=data_collator, tokenizer=tokenizer, 
                      compute_metrics=partial(compute_metrics, metric=metric, label_list=classlabel.names))

    print('Predicting on test samples...')
    predictions, label_ids, _ = trainer.predict(tokenized_datasets['test'])
    predictions = predictions.argmax(axis=2)
    true_predictions = [
        [p for p, i in zip(prediction, label_id) if i != -100]
        for prediction, label_id in zip(predictions, label_ids)]

    return true_predictions

In [ ]:
model_checkpoint = 'test_ner/checkpoint-8/'

tokenizer = create_tokenizer(model_checkpoint=model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
metric = load_metric('seqeval')
# metric = load_metric('/root/.cache/huggingface/modules/datasets_modules/metrics/seqeval/ec5b7242a8c40468d189ca0b2b10612578dbcad311b2a134c99e3ded58a0d6e3/seqeval.py')

In [ ]:
# Exporting the cached metric 

# %cd /root/.cache
# ! zip -r huggingface_cache.zip huggingface/modules/datasets_modules/metrics/seqeval/ec5b7242a8c40468d189ca0b2b10612578dbcad311b2a134c99e3ded58a0d6e3/
# %cd 

In [ ]:
bert_outputs = ner_predict(test_rows, tokenizer=tokenizer, model=model, metric=metric)
print(bert_outputs[0])

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-459a66eaa080ee59/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.
Tokenizing testset...



Creating data collator...
Creating (dummy) training arguments...
Creating trainer...
Predicting on test samples...


/opt/conda/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# predict_batch = 100

# bert_outputs = []
# for i in range(0, len(test_rows), predict_batch):
#     print(f'\rPredicting on samples {i} to {i + predict_batch}...', flush=True)
#     b = ner_predict(test_rows[i:i + predict_batch], model=model, tokenizer=tokenizer, metric=metric)
#     bert_outputs.append(b)

In [ ]:
#export
def get_bert_dataset_labels(test_rows, paper_length, bert_outputs, classlabel=None):
    '''
    Returns:
        bert_dataset_labels (list): Each element is a set consisting of labels predicted
            by the model.
    '''
    test_sentences = [list(zip(*row))[0] for row in test_rows]
#     test_sentences = [row['tokens'] for row in test_rows]
    
    bert_dataset_labels = [] # store all dataset labels for each publication

    for length in paper_length:
        labels = set()
        for sentence, pred in zip(test_sentences[:length], bert_outputs[:length]):
            curr_phrase = ''
            for word, tag in zip(sentence, pred):
                if tag == classlabel.str2int('B'): # start a new phrase
                    if curr_phrase:
                        labels.add(curr_phrase)
                        curr_phrase = ''
                    curr_phrase = word
                elif tag == classlabel.str2int('I') and curr_phrase: # continue the phrase
                    curr_phrase += ' ' + word
                else: # end last phrase (if any)
                    if curr_phrase:
                        labels.add(curr_phrase)
                        curr_phrase = ''
            # check if the label is the suffix of the sentence
            if curr_phrase:
                labels.add(curr_phrase)
                curr_phrase = ''

        # record dataset labels for this publication
        bert_dataset_labels.append(labels)

        del test_sentences[:length], bert_outputs[:length]
        
    return bert_dataset_labels

In [ ]:
classlabel = get_ner_classlabel()
sentences = ['They do not present all the features', 
             'Despite the pretraining on the Tigers EcoNAX dataset',
             'Weirdly there has been lots of studies based on WGS Equality Definitiveness Dataset']
paper_length = [2, 1]
test_rows = [[(word, 0) for word in sentence.split()] for sentence in sentences]
bert_outputs = [[0, 0, 0, 2, 1, 1, 0],
                [0, 0, 0, 0, 0, 2, 1, 1],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1]]
for i, row in enumerate(test_rows):
    assert len(row) == len(bert_outputs[i])

bert_dataset_labels = get_bert_dataset_labels(test_rows, paper_length, bert_outputs, classlabel=classlabel)
print(bert_dataset_labels)

[{'Tigers EcoNAX dataset', 'present all the'}, {'WGS Equality Definitiveness Dataset'}]


In [ ]:
#export
def filter_bert_labels(bert_dataset_labels):
    '''
    When several labels for a paper are too similar, keep just one of them.
    '''
    filtered_bert_labels = []

    for labels in bert_dataset_labels:
        filtered = []

        for label in sorted(labels, key=len):
            label = clean_training_text(label, lower=True)
            if len(filtered) == 0 or all(jaccard_similarity(label, got_label) < 0.75 for got_label in filtered):
                filtered.append(label)

        filtered_bert_labels.append('|'.join(filtered))
    return filtered_bert_labels

In [ ]:
bert_dataset_labels = [{'moma artists catalogue', 'moma artists', 'housing market'},
                       {'rhs flowers fertiliser index', 'deep sea rock salts', 'rhs fertiliser index'}]

filter_bert_labels(bert_dataset_labels)

['moma artists|housing market|moma artists catalogue',
 'deep sea rock salts|rhs fertiliser index']

# Literal matching

In [ ]:
#export
def create_knowledge_bank(pth):
    '''
    Args:
        pth (str): Path to meta data like 'train.csv', which
        needs to have columns: 'dataset_title', 'dataset_label', and 'cleaned_label'.
        
    Returns:
        all_labels (set): All possible strings associated with a dataset from the meta data.
    '''
    df = load_train_meta(pth, group_id=False)
    all_labels = set()
    for label_1, label_2, label_3 in df[['dataset_title', 'dataset_label', 'cleaned_label']].itertuples(index=False):
        all_labels.add(str(label_1).lower())
        all_labels.add(str(label_2).lower())
        all_labels.add(str(label_3).lower())
    return all_labels

In [ ]:
pth = Path('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
all_labels = create_knowledge_bank(pth)
print(len(all_labels))
print(sorted(all_labels)[:10])

180
['2019 ncov complete genome sequences', '2019 ncov genome sequence', '2019 ncov genome sequences', '2019-ncov complete genome sequences', '2019-ncov genome sequence', '2019-ncov genome sequences', 'adni', 'advanced national seismic system (anss) comprehensive catalog (comcat)', 'advanced national seismic system anss comprehensive catalog comcat ', 'advanced national seismic system comprehensive catalog']


In [ ]:
#export
def literal_match(paper, all_labels):
    '''
    Args:
        paper ()
    '''
    text_1 = '. '.join(section['text'] for section in paper).lower()
    text_2 = clean_training_text(text_1, lower=True, total_clean=True)
    
    labels = set()
    for label in all_labels:
        if label in text_1 or label in text_2:
            labels.add(clean_training_text(label, lower=True, total_clean=True))
    return labels

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/test/', sample_submission.Id)

pth = Path('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
all_labels = create_knowledge_bank(pth)

literal_preds = []
for paper_id in sample_submission.Id:
    paper = papers[paper_id]
    literal_preds.append('|'.join(literal_match(paper, all_labels)))
    
literal_preds

['alzheimer s disease neuroimaging initiative adni|adni',
 'common core of data|trends in international mathematics and science study|nces common core of data',
 'sea lake and overland surges from hurricanes|noaa storm surge inundation|slosh model',
 'rural urban continuum codes']

# Overall prediction for submission

In [ ]:
#export
def combine_matching_and_bert(literal_preds, filtererd_bert_labels):
    '''
    For a given sentence, if there's a literal match, use that as the final
    prediction for the sentence.  If there isn't a literal match,
    use what the model predicts.
    '''
    final_predictions = []
    for literal_match, bert_pred in zip(literal_preds, filtered_bert_labels):
        if literal_match:
            final_predictions.append(literal_match)
        else:
            final_predictions.append(bert_pred)
    return final_predictions

In [ ]:
literal_preds = ['mongolian racing cars|reallife headphones', '']
filtered_bert_labels = ['data|dataset', 'hifi dataset|headphones collection data']
combine_matching_and_bert(literal_preds, filtered_bert_labels)

['mongolian racing cars|reallife headphones',
 'hifi dataset|headphones collection data']

In [ ]:
classlabel = get_ner_classlabel()

print('Preparing NER inference data...')
sample_submission = pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
papers = load_papers('/kaggle/input/coleridgeinitiative-show-us-the-data/test/', sample_submission.Id)
test_rows, paper_length = get_ner_inference_data(papers, sample_submission, classlabel=classlabel,
                                                 sentence_definition='section', max_length=512, overlap=20)

print('Loading model, tokenizer, and metric...')
model_checkpoint = 'test_ner/checkpoint-8/'
tokenizer = create_tokenizer(model_checkpoint=model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
metric = load_metric('seqeval')

print('Predicting on each sentence...')
bert_outputs = ner_predict(test_rows, tokenizer=tokenizer, model=model, metric=metric)

print('Getting predicted labels for each article...')
bert_dataset_labels = get_bert_dataset_labels(test_rows, paper_length, bert_outputs, classlabel=classlabel)

print('Keeping just one of labels that are too similar to each other...')
filtered_bert_labels = filter_bert_labels(bert_dataset_labels)

sample_submission['PredictionString'] = filtered_bert_labels

sample_submission.to_csv('submission.csv', index=False)

Preparing NER inference data...
total number of sentences: 105
Loading model, tokenizer, and metric...
Predicting on each sentence...


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-f2fa7b31b822d806/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.
Tokenizing testset...



Creating data collator...
Creating (dummy) training arguments...
Creating trainer...
Predicting on test samples...


Getting predicted labels for each article...
Keeping just one of labels that are too similar to each other...


In [ ]:
sample_submission.head()

,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,
1,2f392438-e215-4169-bebf-21ac4ff253e1,
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,


# Reference
- https://www.kaggle.com/tungmphung/pytorch-bert-for-named-entity-recognition/notebook
- https://www.kaggle.com/tungmphung/coleridge-matching-bert-ner/notebook
- https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb
- https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin
- https://huggingface.co/docs/datasets/loading_metrics.html